# Day 14:  Space Stoichiometry

https://adventofcode.com/2019/day/14

In [1]:
#infile = "./data/day14test1.txt" # ORE needed = 31
#infile = "./data/day14test2.txt" # ORE needed = 165
#infile = "./data/day14test3.txt" # ORE needed = 13312
#infile = "./data/day14test4.txt" # ORE needed = 180697
#infile = "./data/day14test5.txt" # ORE needed = 2210736
infile = "./data/input14.txt"

In [2]:
with open(infile) as f:
    lines = [l.rstrip('\n') for l in f]
#lines

## Part 1

Starting similarly to what I did in the previous (failed) attempt, but now making the reaction dictionary more explicit using multiple keys:

In [3]:
def getReactions(lines):
    R = {}
    for l in lines:
        s = l.split(" => ")
        out = s[1].split(" ")
        material = out[1]
        produced = int(out[0])
        D = {}
        for ingr in s[0].split(", "):
            i = ingr.split(" ")
            ingredient = i[1]
            amount = int(i[0])
            D[ingredient] = { 'ingredient': ingredient, 'amount': amount}
        R[material] = { 'produced': produced , 'ingredients': D} 
    return R

R = getReactions(lines)
R['FUEL']['produced']
#R['A']['ingredients']['ORE']['amount']

1

In my previous attempt I was trying to use temporary dictionary to store how much of each ingredients I had at each step. This was adding the complication of having to check the existence of each ingredient key each time. I discovered the existence of `defaultdict` that seems to solve this problem: a `defaultdict` will never raise a `KeyError`, any key that does not exist gets the value returned by the default factory value.

In [4]:
from collections import defaultdict

available = defaultdict(int)
available['FUEL']

0

In my previous attempt I was lacking a flexible structure to store the ingredients to be produced at each step, and to be freed when something was produced (I was trying to use a simple dictionary, with mixed results and the difficulties of replacing it in the loop). Now I'm trying to use a `Queue()` object to store and retrieve dictionary of ingredients to be produced at each step. The advantage is that a `Queue()` gets freed when I retrieve something (so no need to empty it explicitely)

In [5]:
from queue import Queue

prod = Queue()
P = { "ingredient": 'FUEL', "amount": 1}
prod.put(P)
Q = prod.get()
print(Q)
print(prod.empty())

{'ingredient': 'FUEL', 'amount': 1}
True


In [6]:
from math import ceil

def oreNeeded(P):

    available = defaultdict(int)
    prod = Queue()
    prod.put(P)
    n_ore = 0

    while(not prod.empty()):
        prodstep = prod.get() # queue get freed from the production diction in use
        ## print("Now producing",prodstep['amount'],prodstep['ingredient'])
        # if reached the need for ORE, summing up how much I need. No new order to added to the queue
        if (prodstep['ingredient']=='ORE'):
            n_ore += prodstep['amount']
        # if I have enough leftover from previous production step, just use them for current step
        # (I was not really doing this in my previous attempt...)
        elif prodstep['amount'] <= available[prodstep['ingredient']]:
            available[prodstep['ingredient']] -= prodstep['amount']
        # otherwise produce how much is needed
        else:
            # compute how much is needed given the leftovers from previous step, if any
            needed = prodstep['amount'] - available[prodstep['ingredient']]
            # get the reaction recipe
            reaction = R[prodstep['ingredient']]
            # compute the number of times the reaction need to be perfomed to get the needed amount
            multiplier = ceil( needed / reaction['produced'] )
            # make a new dictionary for each ingredient, add to production queue
            for ingredient in reaction['ingredients']:
                amount = reaction['ingredients'][ingredient]['amount']
                P = { "ingredient": ingredient, "amount": multiplier * amount }
                prod.put(P) # new production dictionary added to production queue
            # compute and store leftovers
            leftover = multiplier * reaction["produced"] - needed
            available[prodstep['ingredient']] = leftover
    return n_ore

tobeproduced = 'FUEL'
amount = 1
P = { "ingredient": tobeproduced, "amount": amount}
n_ore = oreNeeded(P)
print("Needed ORE =", n_ore)

Needed ORE = 136771


### Comments on Part 1

The clear problem I had in my previous attempt, apart from the techincal difficulties in handling the queue of production using a simple dictionary (this lead to too many nested loop, while now I only have one), was the **handling of the leftover ingredients** that were not properly computed, stored and checked when something was needed in a further production step.

#### Python techniques I learned

* Complex structure in dictionaries, using multiple keys
* Use of `defaultdict`
* `Queue()` class

## Part 2

_After collecting ORE for a while, you check your cargo hold: 1 trillion (1000000000000) units of ORE. Given 1 trillion ORE, what is the maximum amount of FUEL you can produce? With that much ore, given the examples above:_

* _The 13312 ORE-per-FUEL example could produce 82892753 FUEL._
* _The 180697 ORE-per-FUEL example could produce 5586022 FUEL._
* _The 2210736 ORE-per-FUEL example could produce 460664 FUEL._

Unless I want to reverse the production sequence (and I definitively do not want to attempt this!), this looks like a **dicotomic search** problem: run the production for representative value of FUEL, get needed ORE, change FUEL accordingly to close in to target ORE value.

Note to self: I discovered that what I call "dicotomic search" from my Univeristy memory (physicist jargon? Italian?) is currently called **binary search**.

In [7]:
tobeproduced = 'FUEL'
targetore = 1000000000000

istep = 0
imax = 100

# set some extremes to initialize the search
amountmin = 0
amountmax = 1000000000000
amount = int((amountmax + amountmin)/2)

while(True):
    
    P = { "ingredient": tobeproduced, "amount": amount}
    n_ore = oreNeeded(P)
    #print('FUEL =', int(amount), 'ORE =',n_ore)

    current = amount
    if n_ore > targetore: # too high, move to lower values
        amount = int((amountmin + amount)/2)
        amountmax = current
    else: # too low, move to lower values
        amount = int((amount + amountmax)/2)
        amountmin = current

    # given the definition of the problem, I should always reach the solution from below
    # (e.g amountmin==amount)
    if amountmin==amount or amountmax==amount: 
        print(amountmin,"***",amount,"***",amountmax)
        break

8193614 *** 8193614 *** 8193615
